# Complexity Level 2

In [1]:
# importing appropriate packages
import networkx as nx #This has to be networkx v1.x not 2.x, but small modifications should allow for verion 2.x
import random
import numpy as np
import time
import V10_dating_function_CL2 as df
import pandas as pd
from datetime import datetime
# define the class
class complexity_level_2:
    def entry_method(self,
                    size,sim_times,amount_of_days,
                    probability, max_swipes, age_range, ages,
                    prob_males,prob_m_likes,prob_f_likes):
        
        
        #----------------------------------Initializing the tracing variables-------------------------#
        # initialize lists which grab the averages at the end of each simulation
        # they populate by running the simulation multiple times by changing the "sim_times" variable
        # populating average amount of matches, lists, dislikes, males, females, and messages for future processing
        total_match_list = list()
        total_like_list = list()
        total_mess_like_list = list()
        total_dislike_list = list()
        total_male_list = list()
        total_female_list = list()
        total_message_list = list()
        
        # creating temporary variables to reset the values
        prob_m_likes_reset = prob_m_likes.copy()
        prob_f_likes_reset = prob_f_likes.copy()
        
        
        # starting the simulation
        for m in range(sim_times):
            #We start off with a network called "Origin Network" that's basically an ER model. It is Directed.
            #with a set initial size of 0 and probability of 0 for edge formation.
            origin_network = nx.erdos_renyi_graph(0, probability, directed=True,  seed=1234567) #setting a seed to ensure reproducability
            
        

            #----------------------------------Initializing the probabilities types for the network-------------------------#
            # variable "runs" does not do anything. It is from Dr. Shaheen's code. We do not use it.
            runs = 0

            # possible genders for nodes in network
            gender_options = ["Male","Female"] 
            
            # the amount of males and females differ, so their probabilties are defined here which are
            # used when initializing our network 
            gender_probabilities = [prob_males, 1-prob_males] # proportion of agents that are male and female


            # female and male lists for finding a partner
            # this list helps with the "potential_swipe" function in the dating functions file
            female_list = []
            male_list = []
            
            
            
            
            #----------------------------------initializing the nodes of the network-------------------------#    
            # this for loop sets the node attributes
            for i in range(size):


                # randomly select gender for node based on gender probabilities
                # randomly select gender for node based on gender probabilities
                gender_randomized = np.random.choice(gender_options, p = gender_probabilities) 


                # appending the male_list if gender randomly selected is male
                if gender_randomized == "Male":
                    male_list.append(i)
                # appending the female list if gender randomly selected is female
                else:
                    female_list.append(i)
                
                # adding the node to the network
                # Populate the network on node at a time. 
                # the node will have the attributes gender, attractiveness, and age
                origin_network.add_node(i, 
                                        {

                                            'gender': gender_randomized,
                                            'physical_attractiveness':random.random(),
                                            'age': np.random.randint(19,ages),
                                            'ethnicity': np.random.randint(1,5)
                                        })
                
                
            #----------------------------------Initializing the edge attributes of the network-------------------------#
            # setting the edge attributes
            # the edge attribute will have all the different types of actions that can be associated with the node
            # it also has match_power and time
            # type and time_order were made by Dr. Shaheen's model. Not going to remove them.
            for edges in origin_network.edges():
                nx.set_edge_attributes(origin_network,'Time', 0)
                nx.set_edge_attributes(origin_network, 'Type', "ER")
                nx.set_edge_attributes(origin_network, 'time_order', 0)
                nx.set_edge_attributes(origin_network, 'message', 0 )
                nx.set_edge_attributes(origin_network, 'like', 0)
                nx.set_edge_attributes(origin_network, 'like_message', 0)
                nx.set_edge_attributes(origin_network, 'dislike', 0)
                nx.set_edge_attributes(origin_network, 'match', 0)
                nx.set_edge_attributes(origin_network, 'match_power',0)
            


            # name and time_count were in Dr. Shaheen's code. We do not use them.
            name = size + 1
            time_count= size + 1
            # "Time order, and time will be used for visualization techniques later" - Dr. Shaheen
            
            
            
            #---------------------------------Initializing Constants/ empty lists--------------------------------------------#

            
             
            # initializing counts
            male_like_count = 0 # counts the number of likes made by male nodes
            female_like_count = 0 # counts the number of likes made by female nodes
            male_like_w_message_count = 0 # counts the number of likes with messages made by male nodes
            female_like_w_message_count = 0 # counts the number of likes with messages made by female nodes
            dislikes = 0
            match = 0 # counts the number of matches generated in model

            
            
            # need to establish an empty list for like_record
            # after someone has liked someone, the edge will appear here for match purposes. Records all instance where random node_a liked random node_b
            like_record = list()

            #make sure to grab node that has sent message with like. 
            # Records all instances where random node_a has sent a message with a like to random node_b/ 
            mess_like_record = list()

            # establish a dislike record to ensure that the individuals who have disliked each other do not try and interact a second time.  
            # if they have already passed on each other. This will ensure that we do not repeat.
            dislike_record = list()
            
            # reseting the values for liking
            prob_m_likes = prob_m_likes_reset.copy()#[0.6,0.8,0.05,0.10]
            prob_f_likes = prob_f_likes_reset.copy()#[0.6,0.8,0.03,0.06]
            
            # creating an empty list for match_records. This is created because within our match records, we need to see who has matched with who.
            match_records = list()

            
            
            # keeping track of the like score for the percentiles later
            list_score_m = [] # establishes the list like score values generated from males nodes to female nodes. Used for messaging interaction. 
            list_score_f = [] # establishes the list like score values generated from females nodes to male nodes.
            
            
            # grabbing the node attributes for later usage in the code
            # essentially, using this code allows us to grab the gender,
            # physical_attractiveness, ect. later in the code
            gender = nx.get_node_attributes(origin_network,'gender')
            physical_attractiveness = nx.get_node_attributes(origin_network,'physical_attractiveness')
            age = nx.get_node_attributes(origin_network,'age')
            ethnicity = nx.get_node_attributes(origin_network,'ethnicity')
            
            




            #----------------------------------Starting the simulation-------------------------#

            for i in range(amount_of_days): # iterating through the days set 
                
                #----------------------------------swiping interactions part of simulation------------------------#

                # swipe_number refers to the iteration for swipes 
                # max_swipes * size gives the average agend max_swipes
                for swipe_number in range(max_swipes*size):

                    # grab one node
                    node_a = random.choice(nx.nodes(origin_network))


                    # getting node_a attribtues
                    gender_a = gender[node_a]
                    age_a = age[node_a]
                    physical_attractiveness_a = physical_attractiveness[node_a]
                    ethnicity_a = ethnicity[node_a]

                    # grabbing node b, a candidate for node a to match with
                    node_b = df.potential_swipe(node_a, 
                                                gender_a, age_a,
                                                male_list, female_list, 
                                                origin_network, age_range,
                                                like_record, dislike_record) 


                    # continuing based on too many fails in the function potential_swipe()
                    # essentially, if that node has already swiped on all of the opposite gender people in the area,
                    # then there will be no more people that agent_a can match with, and therefor we have to go to 
                    # the beginning of the swiping for loop.
                    if node_b == False:
    #                     print("Too many failed attempts")
                        continue


                    # getting attributes for b
                    gender_b = gender[node_b]
                    age_b = age[node_b]
                    physical_attractiveness_b = physical_attractiveness[node_b]
                    ethnicity_b = ethnicity[node_b]


                    # calculating like score of node a to b and vice versa
                    origin_network, like_a,like_b, list_score_m, list_score_f = df.match_power(node_b, node_a, 
                                                                                              gender_a, age_a, 
                                                                                              physical_attractiveness_a,ethnicity_a,ethnicity_b,
                                                                                              gender_b, age_b, physical_attractiveness_b,
                                                                                              origin_network,runs,i,
                                                                                              list_score_m,list_score_f)

                    # Action2 checks of node_b already liked node_a. If so, probability of liking increases. 
                    # If node b liked node a and sent a message with the like, probability of liking increases even more. 
                    # discerning whether a like, or like with message is generated from node a to node b based on whether random number generated satisfied probabilities and match_power thresholds are met
                    origin_network, like_record, dislike_record, mess_like_record,male_like_count, female_like_count,male_like_w_message_count,female_like_w_message_count,dislikes = df.Action2(node_a, node_b, 
                                                                                                                    gender_a, gender_b, like_a, like_b, 
                                                                                                                    physical_attractiveness_a,physical_attractiveness_b,
                                                                                                                    origin_network,like_record,dislike_record,
                                                                                                                    mess_like_record, runs,i,
                                                                                                                    male_like_count,female_like_count,male_like_w_message_count,female_like_w_message_count, prob_m_likes.copy(), prob_f_likes.copy(), dislikes)


                    # now to test for matches
                    # if the list [node b, node a] are in the like_record list, and the other way around is in the 
                    # like_record list, then that means that there is a match!
                    if (([node_b, node_a] in like_record) and ([node_a,node_b] in like_record) and ([node_a,node_b] not in match_records) and ([node_b,node_a] not in match_records)):
            
                        # add the two nodes to the match records
                        # this adds a match link between node_a and node_b, node_b and node_a
                        # essentially, it is an "undirected" edge
                        origin_network, match_records, match = df.Match(node_a,node_b,gender_a, age_a, 
                                                                                                age_b,physical_attractiveness_a,physical_attractiveness_b,
                                                                                                origin_network,match_records, runs, 
                                                                                                i, match)
                        

                    
            print("\n\n\n\n\n")
            print("The amount of matches: {}".format(match))

            print("Message Block is starting now")
            
            # starting the messages
            # generating messages based on match_power probabilities
            origin_network, message_count, message_list = df.message(origin_network,match,list_score_m,list_score_f,runs,i,match_records,gender,age,physical_attractiveness)

            # printing general information about the simulation that was just ran
            print("there are {} Males and {} Females".format(len(male_list),len(female_list)))
            print("the males liked the females a total of {} times \nthe males liked with a message the females a total of {} times \nthe females liked the males a total of {} \nthe females liked with a message the males a total of {} times ".format(male_like_count,male_like_w_message_count, female_like_count,female_like_w_message_count ))
            print("there are {} in the like record and {} in the dislike record.".format(len(like_record),len(dislike_record)))
            print("The message count is {} ".format(message_count))
            
            # appending the counts of each action/agent
            total_match_list.append(match)
            total_like_list.append(len(like_record))
            total_mess_like_list.append(len(mess_like_record))
            total_dislike_list.append(len(dislike_record))
            total_male_list.append(len(male_list))
            total_female_list.append(len(female_list))
            total_message_list.append(message_count)
            
        # zipping the counts so that we can put it in our dataframe
        zippedList =  list(zip(total_match_list, total_like_list,total_mess_like_list, total_dislike_list, total_male_list, total_female_list, total_message_list))
        # assigning the zippedList to a dataframe
        counts_df = pd.DataFrame(zippedList, columns = ['matches' , 'likes','like_with_message', 'dislikes','males','females','messages']) 
        
        
        # exporting the dataframe and the network. This will allow us to do analysis later.
        
        # this is what Dr. Shaheen used to output his network. It does
        # not run for us, but it does have good formatting that we can use
        # nx.write_gexf(origin_network, "sim_" + str(sim_number) + "run" + str("___") + "iter" + str(time.time()) + ".gexf")
        
        
        return origin_network, counts_df

In [2]:
# running the model and setting the parameters
if __name__ == '__main__':
    
    Simulation_times = 1 # the amount of times that you would like to rerun the entire simulation
    size = 100 # Initial size of the network
    sim_times = 5 # amount of times that you would like to run the simulation
    amount_of_days = 2 # defines how many days you want the simulation to run
    max_swipes = 10 # amount of swipes per day
    age_range = 10 # +/- this age for potential swipes/matches
    ages = 21 # number of ages 
    prob_males = 0.68 # proportion of those in the population who are males
    imp_m = 0.73 # initial message prob for males
    
    probability = 0.000 # intializing probability of the erdos renyi network /// should keep this at 0 for our simulations
    
    
    # typical default values: 0.6,0.8,0.05,0.1
    high_like_male_prob = 0.6 # if the male is greatly attracted to the female, this is the probability that the male will like
    high_like_w_message_male_prob = 0.2 # if the male is greatly attracted to the female, this is the probability that the male will like with a message
    low_like_male_prob = 0.05 # if the male is NOT greatly attracted to the female, this is the probability that the male will like
    low_like_w_message_male_prob = 0.07 # if the male is NOT greatly attracted to the female, this is the probability that the male will like with a message
    
    # typical default values: [0.6,0.8,0.03,0.06]
    high_like_female_prob = 0.6 # if the female is greatly attracted to the female, this is the probability that the male will like
    high_like_w_message_female_prob = 0.2 # if the female is greatly attracted to the male, this is the probability that the female will like with a message
    low_like_female_prob = 0.03 # if the female is NOT greatly attracted to the male, this is the probability that the male will like
    low_like_w_message_female_prob = 0.05 # if the female is NOT greatly attracted to the male, this is the probability that the female will like with a message
    
    # adding the probabilities into the list for the function: Action2
    prob_m_likes = [high_like_male_prob,high_like_male_prob + high_like_w_message_male_prob,
                    low_like_male_prob,low_like_w_message_male_prob+low_like_male_prob]
    prob_f_likes = [high_like_female_prob,high_like_w_message_female_prob+high_like_female_prob,
                    low_like_female_prob,low_like_w_message_female_prob+low_like_female_prob]
    
    global max_size
    max_size = 20 # set this value to set all other relevant parameters. The model will ouput a network roughly at this size
#     entry_iterations = int(amount_of_days * 0.3) # used to iterate actions for entry only.
n = 1
# self, size, probability, max_swipes, prob_female_likes, prob_female_super_likes, prob_male_likes, prob_male_super_likes, prob_males, prob_ori_m, prob_ori_f):

while n <= Simulation_times:
    print ("simulation starts...")
    A = complexity_level_2() #iterations is always set at 2 for now
    B, counts_df = A.entry_method(
                    size,sim_times,amount_of_days,
                    probability, max_swipes, age_range, ages,
                    prob_males,prob_m_likes,prob_f_likes
                  )
    print("Simulation Concluded")
    n += 1
    
    

simulation starts...






The amount of matches: 8
Message Block is starting now
there are 72 Males and 28 Females
the males liked the females a total of 102 times 
the males liked with a message the females a total of 63 times 
the females liked the males a total of 25 
the females liked with a message the males a total of 22 times 
there are 212 in the like record and 1755 in the dislike record.
The message count is 2 






The amount of matches: 5
Message Block is starting now
there are 70 Males and 30 Females
the males liked the females a total of 94 times 
the males liked with a message the females a total of 65 times 
the females liked the males a total of 33 
the females liked with a message the males a total of 14 times 
there are 206 in the like record and 1770 in the dislike record.
The message count is 3 






The amount of matches: 7
Message Block is starting now
there are 72 Males and 28 Females
the males liked the females a total of 85 times 
the males liked with a mes

# Exporting the file

In [3]:
# telling the time
today = datetime.now()
month = today.month
day = today.day
year = today.year
hour = today.hour
minute = today.minute


# creating the parameters_df
parameters_used = ["complexity_level","date","time","sim_times", "amount_of_days", "max_swipes", "age_range", "age","high_like_male_prob","low_like_male_prob","high_like_female_prob","low_like_female_prob"]
parameter_values = [["2","{}/{}/{}".format(month,day,year),"{}:{}".format(hour,minute),sim_times, amount_of_days, max_swipes, age_range, ages + 1, high_like_male_prob, low_like_male_prob, high_like_female_prob, low_like_female_prob]]
parameters_df = pd.DataFrame(parameter_values, columns = parameters_used)


# exportint the parameters and the count_df
parameters_df.to_csv('CL2 DATE {} MON {} DAY {} YEAR TIME {} HR {} MIN PARAMETERS.csv'.format(month,day,year,hour,minute))
count_df.to_csv("CL2 DATE {} MON {} DAY {} YEAR TIME {} HR {} MIN COUNTS.csv".format(month,day,year,hour,minute))

# exporting the last network of the simulation itself


# Extracting the network itself

In [4]:
# to visualize in r
nx.write_edgelist(B,'test.csv',data=['like','dislike','match','message','like_score'])
# need to read the dataframe back in 
edge_list_df = pd.read_csv("test.csv",names=["node_a","node_b","like","dislike",'match','message','like_score'],sep=' ')
# B.get_edge_data()
# initialize the lists to keep track of genders 
gender_list = list()
physical_attractiveness_list = list()
age_list = list()
ethnicity_list = list()

# be able to retreive the genders,attractiveness,ages, and the ethnicities of the nodes
gender = nx.get_node_attributes(B,'gender')
physical_attractiveness = nx.get_node_attributes(B,'physical_attractiveness')
age = nx.get_node_attributes(B,'age')
ethnicity = nx.get_node_attributes(B,'ethnicity')


for node_a in edge_list_df['node_a']:
    
    
    
    
    gender_list.append(gender[node_a])
    physical_attractiveness_list.append(physical_attractiveness[node_a])
    age_list.append(age[node_a])
    ethnicity_list.append(ethnicity[node_a])
    
    
node_a_list = edge_list_df["node_a"].to_list()
node_b_list = edge_list_df["node_b"].to_list()
like_list = edge_list_df["like"].to_list()
dislike_list = edge_list_df["dislike"].to_list()
match_list = edge_list_df["match"].to_list()
message_list = edge_list_df["message"].to_list()
like_score_list = edge_list_df["like_score"].to_list()
zipped_list = list(zip(node_a_list,node_b_list,like_list,dislike_list,match_list,message_list,like_score_list,gender_list,physical_attractiveness_list,age_list,ethnicity_list))
edge_att_df = pd.DataFrame(zipped_list, columns =['node_a', 'node_b','like','dislike','match','message','like_score','gender_node_a','physical_attractiveness_node_a','age_node_a','ethnicity_node_a'])
edge_att_df.to_csv("CL2 DATE {}M {}D {}Y TIME {}hr {}min NETWORK.csv".format(month,day,year,hour,minute))
print("done")

In [5]:
#     probability = 0.000 # intializing probability of the erdos renyi network // should keep this at 0 for our simulations
#     amount_of_days = 3 # defines how many days you want the simulation to run
#     max_swipes = 5 # amount of swipes per day
#     prob_female_likes = 0.25 # probability that the female likes the male
#     prob_female_mess_likes = 0.40# probability that the female likes the male and sends message with it
#     prob_male_likes = 0.50 # probability that the male likes the female
#     prob_male_mess_likes = 0.10# probability that the male likes the female and sends message with it
#     prob_males = 0.68 # proportion of those in the population who are males
#     Simulation_times = 1 # how many times would you like to run the simulation
#     imp_m = 0.73 # initial message prob for males
#     age_range = 5 # +/- this age for potential swipes/matches
#     ages = 50 # number of ages 
#     sim_times = 3 # amount of times to run simulation
#     global max_size
#     max_size = 20 # set this value to set all other relevant parameters. The model will ouput a network roughly at this size
# #     entry_iterations = int(amount_of_days * 0.3) # used to iterate actions for entry only.
# n = 1

# typical default values: 0.6,0.8,0.05,0.1